In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression

In [4]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [5]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [6]:
lr = LinearRegression(featuresCol='features', labelCol='label',
                     predictionCol='prediction')

In [7]:
lrModel = lr.fit(training)

In [8]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [9]:
lrModel.intercept

0.14228558260358093

In [10]:
training_summary = lrModel.summary

In [11]:
training_summary.rootMeanSquaredError

10.16309157133015

In [12]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [15]:
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [16]:
train_data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
|-28.046018037776633|(10,[0,1,2,3,4,5,...|
|-26.805483428483072|(10,[0,1,2,3,4,5,...|
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|
|-21.432387764165806|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-19.884560774273424|(10,[0,1,2,3,4,5,...|
|-19.872991038068406|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -19.66731861537172|(10,[0,1,2,3,4,5,...|
|-19.402336030214553|(10,[0,1,2,3,4,5,...|
|-18.845922472898582|(10,[0,1,2,3,4,5,...|
| -18.27521356600463|(10,[0,1,2,3,4,5,...|
|-17.803626188664516|(10,[0,1,2,3,4,5,...|
|-17.494200356883344|(10,[0,1,2,3,4,5,...|
|-17.428674570939506|(10,[0,1,2,3,4,5,...|
| -17.32672073267595|(10,[0,1,2,3,4,5,...|
| -16.71909683360509|(10,[0,1,2,3,4,5,...|
+----------

In [17]:
train_data.describe().show()

+-------+--------------------+
|summary|               label|
+-------+--------------------+
|  count|                 334|
|   mean|-0.35393877533733065|
| stddev|  10.332745302932135|
|    min| -28.571478869743427|
|    max|   27.78383192005107|
+-------+--------------------+



In [18]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                167|
|   mean| 1.4785440172596411|
| stddev| 10.209420280600268|
|    min|-26.736207182601724|
|    max| 26.903524792043335|
+-------+-------------------+



In [19]:
correct_model = lr.fit(train_data)

In [20]:
test_results = correct_model.evaluate(test_data)

In [21]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-22.338840529332803|
| -20.34132589479252|
|-20.756722039752887|
|-14.300340965055156|
|-16.502602093492545|
| -17.32495215448968|
|-21.576669451558423|
| -15.57008484716454|
|-16.008760645372625|
|-13.020087162646282|
| -16.29186749086617|
|-14.079834707144995|
|-12.196744375229933|
|-16.291236693000506|
| -8.358074831749718|
|-13.539040229512127|
| -10.81925207627026|
|-12.722093942740484|
| -6.870904538084627|
|-12.085175113818437|
+-------------------+
only showing top 20 rows



In [23]:
test_results.rootMeanSquaredError

10.552308307825403

In [29]:
### Now we can play with hyperparameters with model to try improve it

In [24]:
### Deploy model

In [25]:
unlabeled_data = test_data.select('features')

In [26]:
predictions = correct_model.transform(unlabeled_data)

In [27]:
predictions.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...| -4.397366653268921|
|(10,[0,1,2,3,4,5,...|-3.1461142261439923|
|(10,[0,1,2,3,4,5,...| 0.5446447807942165|
|(10,[0,1,2,3,4,5,...| -4.867951657908605|
|(10,[0,1,2,3,4,5,...|-0.5627975323834704|
|(10,[0,1,2,3,4,5,...|0.29845989028013153|
|(10,[0,1,2,3,4,5,...|   4.88446243024732|
|(10,[0,1,2,3,4,5,...|-0.5155741938569494|
|(10,[0,1,2,3,4,5,...|0.05724807957805389|
|(10,[0,1,2,3,4,5,...| -2.328783992732971|
|(10,[0,1,2,3,4,5,...|  1.469714581114981|
|(10,[0,1,2,3,4,5,...|-0.6829235457861316|
|(10,[0,1,2,3,4,5,...| -1.223850400660824|
|(10,[0,1,2,3,4,5,...| 3.3690135896300877|
|(10,[0,1,2,3,4,5,...| -4.415152167501479|
|(10,[0,1,2,3,4,5,...| 1.0475981519657132|
|(10,[0,1,2,3,4,5,...|-1.6600281351812376|
|(10,[0,1,2,3,4,5,...|0.25443756170762377|
|(10,[0,1,2,3,4,5,...| -4.769645139804199|
|(10,[0,1,2,3,4,5,...| 1.4850447719094035|
+----------